In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm
from scipy import stats
from scipy.optimize import minimize
from scipy.optimize import curve_fit
from copy import deepcopy
import tensorflow as tf
#from multiprocessing import Pool, current_process
import pickle

from freedom.toy_model import advanced_toy_model

In [ ]:
params = {'legend.fontsize': 17,
          'figure.figsize': (15, 9.3),
          'axes.labelsize': 24,
          'axes.titlesize': 24,
          'xtick.labelsize': 22,
          'ytick.labelsize': 22}
plt.rcParams.update(params)
params = ['x', 'y', 't', 'E', 'azi']

In [ ]:
#detectors = np.vstack([np.repeat(np.linspace(-5, 5, 6), 2), np.tile(np.linspace(-1, 1, 2), 6)]).T
detectors = np.vstack([np.repeat(np.linspace(-10, 10, 10), 10), np.tile(np.linspace(-10, 10, 10), 10)]).T

In [ ]:
toy_experiment = advanced_toy_model.advanced_toy_experiment(detectors=detectors, isotrop=False) #, time_dist=advanced_toy_model.pandel

In [ ]:
def func(X, event, only_c=False, only_h=False, fix=[None]):
    #X: hypo_x, hypo_b, hypo_t, hypo_N_src, hypo_ang
    assert only_c + only_h < 2
    
    if fix[0] != None:
        X = np.insert(X, fix[0], fix[1])
        #X[fix[0]] = fix[1]
    
    pos = np.array([X[0], X[1]])
    c_term = -toy_experiment.charge_term(event[0], pos, X[3], X[4]) #
    h_term = -toy_experiment.hit_term(event[1], pos, X[2], X[3], X[4]) #
    
    if only_c: return c_term
    if only_h: return h_term
    return c_term + h_term
        
bounds = np.array([[-15,15], [-15,15], [-5,5], [0.1,40], [0, 2*np.pi]])

In [ ]:
def gradients2(event, LLH_service, point, D=1e-3):
    llh_center = LLH_service(point, event)
    grads = []
    for i in range(len(point)):
        if i < 2:
            d = 3 #5
        else:
            d = D
        
        p = deepcopy(point)
        p[i] += d
        llh_plus = LLH_service(p, event)
        p[i] -= 2*d
        llh_minus = LLH_service(p, event)
        
        grads.append((llh_plus-2*llh_center+llh_minus)/(d**2))
    
    return np.array(grads)

def gradientsM(event, LLH_service, point, i, j, d=1e-3):
    assert i != j
    p = deepcopy(point)
    p[i] += d
    p[j] += d
    llh_plus_plus = LLH_service(p, event)
    p[j] -= 2*d
    llh_plus_minus = LLH_service(p, event)
    p[i] -= 2*d
    llh_minus_minus = LLH_service(p, event)
    p[j] += 2*d
    llh_minus_plus = LLH_service(p, event)
    
    grad = (llh_plus_plus - llh_plus_minus - llh_minus_plus + llh_minus_minus)/(4*d**2)
    
    return grad

## Test event

In [ ]:
# generate one test event

example_pos_src = np.array([-1, 2.2])
example_N_src = 7
example_ang_src = np.pi
test_event = toy_experiment.generate_event(example_pos_src, N_src=example_N_src, ang_src=example_ang_src)
'''
test_event = events[4]
example_pos_src = Truth[4][:2]
example_N_src = Truth[4][3]
example_ang_src = Truth[4][4]
'''

In [ ]:
#plt.scatter(toy_experiment.detectors[0], toy_experiment.detectors[1], color='grey')
plt.scatter(test_event[0][:, 1], test_event[0][:, 2], s=15*test_event[0][:, 0])
plt.scatter(example_pos_src[0], example_pos_src[1], color='black', marker='$T$', s=70)
#plt.scatter(Truth[3046][0], Truth[3046][1], color='black', marker='$T$', s=70)

#### LLH scans

In [ ]:
# Grid scan
X = np.linspace(-10, 10, 100)
Y = np.linspace(-10, 10, 100)
x, y = np.meshgrid(X, Y)

g = {}
g['hit_terms'] = np.empty(x.shape)
g['charge_terms'] = np.empty(x.shape)

for idx in np.ndindex(x.shape):
    hypo_pos =  np.array([x[idx], y[idx]])
    hypo_t = 0
    hypo_N_src = example_N_src
    hypo_ang_src = example_ang_src
    g['hit_terms'][idx] = -toy_experiment.hit_term(test_event[1], hypo_pos, hypo_t, hypo_N_src, hypo_ang_src)
    g['charge_terms'][idx] = -toy_experiment.charge_term(test_event[0], hypo_pos, hypo_N_src, hypo_ang_src)
    
g['total_llh'] = g['hit_terms'] + g['charge_terms']
g['total_llh'] -= np.min(g['total_llh'])

In [ ]:
#reco event 
points = [[-1, 2.2, 0, 7, np.pi]] #[Truth[4]]
mini = minimize(func, points[0], method='SLSQP', bounds=bounds, args=(np.array(test_event))) #Nelder-Mead
#args = (np.array(test_event), False, False, [None])
#mini2 = minimize(func, points[0], method='SLSQP', bounds=bounds, args=args)

In [ ]:
#plot 2d LLH space
#plt.pcolormesh(X, Y, g['hit_terms']) 
#plt.pcolormesh(X, Y, g['charge_terms'])
plt.pcolormesh(X, Y, g['total_llh']) #, vmax=10
plt.colorbar()
plt.scatter(example_pos_src[0], example_pos_src[1], color='white', marker='$T$', s=70)
plt.scatter(points[-1:][0], points[-1:][1], color='r')
plt.scatter(toy_experiment.detectors[0], toy_experiment.detectors[1], color='black')

#plt.savefig('../../plots/toy_model/xy_scan_2d', bbox_inches='tight')

In [ ]:
# 1d LLH space
#point = np.array([example_pos_src[0], example_pos_src[1], 0, example_N_src, example_ang_src])
point = mini.x #Reco[3046]

X = np.linspace(point[0]-3, point[0]+3, 100)
Y = np.linspace(point[1]-3, point[1]+3, 100)
T, E = np.linspace(point[2]-2, point[2]+2, 100), np.linspace(max(point[3]-4,0.1), point[3]+4, 100)
ranges = [X, Y, T, E]
llhs = []

for i in range(len(ranges)):
    llh, llh_wrong = [], []
    p = deepcopy(point)
    for idx in np.ndindex(ranges[i].shape):
        p[i] = ranges[i][idx]
        llh.append(func(p, test_event)) #events[3046]
    llhs.append(llh-np.min(llh))
    
grads = gradients2(test_event, func, point) #events[3046]
grads

In [ ]:
plt.figure(figsize=(15, 11))
for i in range(4):
    plt.subplot(2,2,i+1)
    plt.plot(ranges[i], llhs[i])
    plt.axvline(point[i], label='Best-fit true llh', color='blue')
    plt.axvline(points[0][i], color='black', linestyle='--', label='Truth')
    plt.plot(ranges[i], grads[i]/2*(ranges[i]-point[i])**2, linestyle='--', label='Curvature')
    
    plt.legend(fontsize=15)
    plt.xlabel(params[i])
    #plt.ylim(0,2)
#plt.savefig('../../plots/toy_model/llh_scans2_wrong', bbox_inches='tight')

## Many events

In [ ]:
N = 3000
events, Truth = toy_experiment.generate_events(N, xlims=(-10,10), blims=(-10,10), N_lims=(1,30))
Truth = np.insert(Truth, 2, 0, axis=1)

In [ ]:
LLHs, Reco, n_hits, Grads = [], [], [], []
for i, event in enumerate(events):
    #n_hits.append(np.sum(event[0][:,0]))
    #print(func(points[0], event))
    #print('-------------')
    
    args = (np.array(event), False, False, [None])
    m = minimize(func, np.random.normal(Truth[i]), method='Nelder-Mead', bounds=bounds, args=args)
    LLHs.append(m.fun)
    Reco.append(m.x)
    Grads.append(gradients2(event, func, m.x[:4]))
    #print(m.fun)
    #print('-------------')
    
LLHs, Reco, n_hits, Grads = np.array(LLHs), np.array(Reco), np.array(n_hits), np.array(Grads)

In [ ]:
fig = plt.figure(figsize=(20, 12))
plt.suptitle('Reco - Truth', size=20, y=.91)

plt.subplot(2,3,1)
x_diff = Reco[:, 0]-Truth[:, 0]
plt.hist(x_diff, np.linspace(-2,2,50))
plt.axvline(0, color='black', linestyle='--')
plt.text(-2, 0.06*N, 'std=%.2f'%(np.std(x_diff)), size=16)
plt.text(-2, 0.03*N, 'iqr=%.2f'%(stats.iqr(x_diff)), size=16)
plt.xlabel('x')

plt.subplot(2,3,2)
y_diff = Reco[:, 1]-Truth[:, 1]
plt.hist(y_diff, np.linspace(-2,2,50))
plt.axvline(0, color='black', linestyle='--')
plt.text(-2, 0.06*N, 'std=%.2f'%(np.std(y_diff)), size=16)
plt.text(-2, 0.03*N, 'iqr=%.2f'%(stats.iqr(y_diff)), size=16)
plt.xlabel('y')

plt.subplot(2,3,3)
t_diff = Reco[:, 2]-Truth[:, 2]
plt.hist(t_diff, np.linspace(-1,1,50))
plt.axvline(0, color='black', linestyle='--')
plt.text(-1, 0.06*N, 'std=%.2f'%(np.std(t_diff)), size=16)
plt.text(-1, 0.03*N, 'iqr=%.2f'%(stats.iqr(t_diff)), size=16)
plt.xlabel('t')

plt.subplot(2,3,4)
E_diff = Reco[:, 3]-Truth[:, 3]
plt.hist(E_diff, np.linspace(-10,10,50))
plt.axvline(0, color='black', linestyle='--')
plt.text(-10, 0.04*N, 'std=%.2f'%(np.std(E_diff)), size=16)
plt.text(-10, 0.02*N, 'iqr=%.2f'%(stats.iqr(E_diff)), size=16)
plt.xlabel('E')

plt.subplot(2,3,5)
a_diff = Reco[:, 4]-Truth[:, 4]
a_diff = np.where(a_diff<-np.pi, a_diff+2*np.pi, a_diff)
a_diff = np.where(a_diff>np.pi, a_diff-2*np.pi, a_diff)
plt.hist(a_diff, np.linspace(-np.pi, np.pi, 50))
plt.axvline(0, color='black', linestyle='--')
plt.text(-np.pi, 0.06*N, 'std=%.2f'%(np.std(a_diff)), size=16)
plt.text(-np.pi, 0.03*N, 'iqr=%.2f'%(stats.iqr(a_diff)), size=16)
plt.xlabel('azi')

#plt.savefig('../../plots/uncertainties/from_dLLH/toy/reco_dists_wrongLLH', bbox_inches='tight') #_truthSeeded

In [ ]:
1/np.sqrt(np.average(Grads,axis=0))

time

In [ ]:
plt.scatter(n_hits, Grads[:, 2])
plt.xlabel('# hits')
plt.ylabel('time curvature')
#plt.savefig('../../plots/toy_model/time_nHits_curvature', bbox_inches='tight')

In [ ]:
bin_edges = np.linspace(1, 175, 41)
bin_center = (bin_edges[:-1]+bin_edges[1:])/2
stds, std_err, means = [], [], []
for i in range(len(bin_center)):
    stds.append(np.std(t_diff[(n_hits<bin_edges[i+1]) & (n_hits>bin_edges[i])]))
    std_err.append(stds[-1]/np.sqrt(len(t_diff[(n_hits<bin_edges[i+1]) & (n_hits>bin_edges[i])])))
    means.append(np.mean(t_diff[(n_hits<bin_edges[i+1]) & (n_hits>bin_edges[i])]))
    #print(len(t_diff[(n_hits<bin_edges[i+1]) & (n_hits>bin_edges[i])]))

In [ ]:
np.sqrt(1/np.average(n_hits[(n_hits<175) & (n_hits>0)])), np.average(np.sqrt(1/n_hits[(n_hits<175) & (n_hits>0)]))

In [ ]:
theo_lim = np.sqrt(1/np.average(n_hits[n_hits<175]))
plt.plot(bin_center, np.sqrt(1/bin_center), label='theoretical limit (%.2f)'%(theo_lim))
plt.errorbar(bin_center, stds, std_err, fmt='o', label='from fit (%.2f)'%(np.std(t_diff[n_hits<175])))
#plt.scatter(bin_center, means)
plt.legend()
plt.xlabel('# hits')
plt.ylabel('time resolution (std)')
plt.ylim(0, 0.6)

#plt.savefig('../../plots/toy_model/time_resolution_nHits', bbox_inches='tight')

In [ ]:
theos, reals, stops = [], [], np.linspace(0, 150, 20)
for s in stops:
    theos.append(np.sqrt(1/np.average(n_hits[(n_hits<175)&(n_hits>s)])))
    reals.append(np.std(t_diff[(n_hits<175)&(n_hits>s)]))

In [ ]:
plt.scatter(stops, theos, label='theoretical limit')
plt.scatter(stops, reals, label='from fit')
plt.legend()
plt.xlabel('from 175 down to # hits considered')
plt.ylabel('time resolution (std)')

#plt.savefig('../../plots/toy_model/time_resolution_nHits_diff', bbox_inches='tight')

energy

In [ ]:
plt.scatter(n_hits/Reco[:, 3]**2, Grads[:, 3])
plt.xlabel('total charge / E²')
plt.ylabel('energy curvature')
plt.xlim(0,4)
plt.ylim(0,4)
#plt.savefig('../../plots/toy_model/energy_qual_curvature', bbox_inches='tight')

In [ ]:
E_qual = n_hits/Reco[:, 3]**2 #Grads[:,3]
bin_edges = np.logspace(-1, 0.6, 41)
bin_center = (bin_edges[:-1]+bin_edges[1:])/2
stds, std_err, means = [], [], []
for i in range(len(bin_center)):
    stds.append(np.std(E_diff[(E_qual<bin_edges[i+1]) & (E_qual>bin_edges[i])]))
    std_err.append(stds[-1]/np.sqrt(len(E_diff[(E_qual<bin_edges[i+1]) & (E_qual>bin_edges[i])])))
    means.append(np.mean(E_diff[(E_qual<bin_edges[i+1]) & (E_qual>bin_edges[i])]))
    #print(len(E_diff[(E_qual<bin_edges[i+1]) & (E_qual>bin_edges[i])]))

In [ ]:
np.sqrt(1/(np.average(E_qual[(E_qual<4)&(E_qual>0)]))), np.average(np.sqrt(1/(E_qual[(E_qual<4)&(E_qual>0)])))

In [ ]:
theo_lim = np.sqrt(1/(np.average(E_qual[(E_qual<4)])))
plt.plot(bin_center, np.sqrt(1/bin_center), label='theoretical limit (%.2f)'%(theo_lim))
plt.errorbar(bin_center, stds, std_err, fmt='o', label='from fit (%.2f)'%(np.std(E_diff[(E_qual<4)])))
#plt.scatter(bin_center, means)
plt.legend()
plt.xlabel('total charge / E²')
plt.ylabel('E resolution (std)') #bias (mean)
plt.ylim(0,3)

#plt.savefig('../../plots/toy_model/E_resolution_', bbox_inches='tight')

xy

In [ ]:
#plt.hist(Grads[:,0],100, alpha=0.5)
#plt.hist(Grads[:,1],100, alpha=0.5)
plt.scatter(Grads[:,0], Grads[:,1])
plt.xlabel('x curvature')
plt.ylabel('y curvature')
#plt.savefig('../../plots/toy_model/xy_curvature_curvature', bbox_inches='tight')

In [ ]:
x_qual = Grads[:,0]
bin_edges = np.linspace(1, 100, 41)
bin_center = (bin_edges[:-1]+bin_edges[1:])/2
stds, std_err, means = [], [], []
for i in range(len(bin_center)):
    stds.append(np.std(x_diff[(x_qual<bin_edges[i+1]) & (x_qual>bin_edges[i])]))
    std_err.append(stds[-1]/np.sqrt(len(x_diff[(x_qual<bin_edges[i+1]) & (x_qual>bin_edges[i])])))
    means.append(np.mean(x_diff[(x_qual<bin_edges[i+1]) & (x_qual>bin_edges[i])]))

In [ ]:
theo_lim = np.sqrt(1/(np.average(x_qual[(x_qual<100)&(x_qual>1)])))
plt.plot(bin_center, np.sqrt(1/bin_center), label='theoretical limit (%.2f)'%(theo_lim))
plt.errorbar(bin_center, stds, std_err, fmt='o', label='from fit (%.2f)'%(np.std(x_diff[(x_qual<100)&(x_qual>1)])))
#plt.scatter(bin_center, means)
plt.legend()
plt.xlabel('x curvature')
plt.ylabel('x resolution (std)')
plt.ylim(0,0.8)

#plt.savefig('../../plots/toy_model/x_resolution_curvature', bbox_inches='tight')

In [ ]:
y_qual = Grads[:,1]
bin_edges = np.linspace(1, 100, 41)
bin_center = (bin_edges[:-1]+bin_edges[1:])/2
stds, std_err, means = [], [], []
for i in range(len(bin_center)):
    stds.append(np.std(y_diff[(y_qual<bin_edges[i+1]) & (y_qual>bin_edges[i])]))
    std_err.append(stds[-1]/np.sqrt(len(y_diff[(y_qual<bin_edges[i+1]) & (y_qual>bin_edges[i])])))
    means.append(np.mean(y_diff[(y_qual<bin_edges[i+1]) & (y_qual>bin_edges[i])]))

In [ ]:
theo_lim = np.sqrt(1/(np.average(y_qual[(y_qual<100)&(y_qual>1)])))
plt.plot(bin_center, np.sqrt(1/bin_center), label='theoretical limit (%.2f)'%(theo_lim))
plt.errorbar(bin_center, stds, std_err, fmt='o', label='from fit (%.2f)'%(np.std(y_diff[(y_qual<100)&(y_qual>1)])))
#plt.scatter(bin_center, means)
plt.legend()
plt.xlabel('y curvature')
plt.ylabel('y resolution (std)')
plt.ylim(0,0.8)

#plt.savefig('../../plots/toy_model/y_resolution_curvature', bbox_inches='tight')

## Fisher info mat

In [ ]:
def FishMatCalculator(event, LLH_service, fit_point):
    N = len(fit_point)

    grads = gradients2(event, LLH_service, fit_point) #, d=.1
    
    FisherMatrix = np.zeros((N,N))
    for i in range(N):
        for j in range(N):
            if i==j: 
                FisherMatrix[i, j] = grads[i]
            else:
                FisherMatrix[i, j] = gradientsM(event, LLH_service, fit_point, i, j)
    
    if np.isnan(FisherMatrix).any() or np.isinf(FisherMatrix).any():
        print('ERROR: Fisher Matrix has invalid values. Continue...')
        return

    f = TikhonovCorrection(FisherMatrix)

    return f

def TikhonovCorrection(FisherMatrix, threshold=0.001):
    if (np.linalg.eigvals(FisherMatrix) < 0).any():
        #print('negative eigenvalues detected...')
        if (np.linalg.eigvals(FisherMatrix) > -threshold).all():
            #print('neg. EV seem quite small. Trying Tikhonov correction...')
            correction = 0
            # find appropriate correction value first:
            for eigval in np.linalg.eigvals(FisherMatrix):
                if eigval < 0 and abs(eigval) > correction:
                    correction = abs(eigval)*2
            for k in range(0,5):
                FisherMatrix[k,k] += correction
            #if (np.linalg.eigvals(FisherMatrix) >= 0).all():
            #    print('...worked!')
    return FisherMatrix

In [ ]:
keep = []
F = np.zeros((4, 4))
for i in range(N):
    event = events[i]
    if np.sum(event[0][:, 0]) <= 4:
        continue
    else:
        keep.append(i)
        
    F += FishMatCalculator(event, func, np.array(Reco[i][:4]))

F = F/len(keep)
cov = np.linalg.inv(F)

In [ ]:
plt.imshow(np.abs(F), norm=LogNorm())
plt.colorbar()
plt.xticks(range(4), ['x', 'y', 't', 'E']) #, 'azi'
plt.yticks(range(4), ['x', 'y', 't', 'E']) #, 'azi'
for i in range(4):
    for j in range(4):
        plt.text(i, j, '%.2f'%(F[i][j]), size=14, ha='center', va='center')
#plt.savefig('../../plots/toy_model/Fisher_mat', bbox_inches='tight')

In [ ]:
plt.scatter(range(4), np.sqrt(cov.diagonal()))
plt.xticks(range(4), ['x', 'y', 't', 'E']) #, 'azi'
#plt.yscale('log')
for i, r in enumerate(np.sqrt(cov.diagonal())): 
    plt.text(i, r, '%.3f'%(r), size=16, ha='center')
#plt.savefig('../../plots/toy_model/Fisher_resolutions', bbox_inches='tight')

In [ ]:
np.sqrt(1/F.diagonal())